## NA correction & Enrichment

In [9]:
import pandas as pd
from itertools import combinations
import glob

# Get Metabolite info file into a pandas dataframe
m_df =pd.read_csv("./MetaboliteInfo_v5.5.csv").drop_duplicates()
m_df = m_df.set_index("compoundId")
# Modify Lower case the headers in Metabolite info df
m_df.columns = m_df.columns.str.lower()
m_df

# Get a list of Area files starting with "A"
a_files = glob.glob("A*.csv")
# Get a list of Sampleinfo files starting with "S"
s_files = glob.glob("S*.csv")

#Functions

def combi(a,b):
    '''
    This function computes the total number of ways to choose b elements from a set of a elements without regard to the order of the elements. 
    It uses the `combinations` function from the itertools module to generate all possible combinations and then returns the length of this list, which represents the total number of combinations.

    Parameters:
    a (int or convertible to int): The total number of elements in the set. Must be a non-negative integer.
    b (int or convertible to int): The number of elements to choose from the set. Must be a non-negative integer and less than or equal to a.

    Returns:
    int: The total number of combinations for choosing b elements from a set of a elements.

    Raises:
    ValueError: If a or b is negative or if b is greater than a.
    '''
    return len(list(combinations(range(int(a)),int(b))))


def natural_abundance(df): #calculates expected M+1 and M+2 NA values 
    """
    Description: 
    
    Parameters:
    df (dataframe)
    
    Returns:
    Series: List of Metabolites with their corresponding natural abundance correction factor
    """
    # Natural abundance of stable isotopes (13C,2H,15N,17O,18O)
    na13C,na2H,na15N,na17O,na18O = 0.0107,0.000115,0.00368,0.00038,0.00205
    # From  the 1997 report of the IUPAC Subcommittee for Isotopic Abundance Measurements 
    # by K.J.R. Rosman, P.D.P. Taylor Pure Appl. Chem. 1999, 71, 1593-1607
    
    L1 = [na13C,na2H,na15N,na17O]
    
    #Natural abundance Calculation
    
    #M+0
    #nC0 - number of carbons from parent molecule
    mM0=df[(df['mass']=='M+0')]
    nC0,nH0,nN0,nO0 = mM0['parent.carbons'],mM0['parent.hydrogen'],mM0['parent.nitrogen'],mM0['parent.oxygen']
    
    L_M0 = [nC0,nH0,nN0,nO0]
    
    naM0 = sum(sum((L1[i]**j)*L_M0[i].apply(lambda x: combi(x,j)) for j in [1,2]) for i in range(len(L1))) \
    + (na18O*nO0) + sum(L1[i]*L1[j]*L_M0[i]*L_M0[j] for i,j in combinations(range(len(L1)),2))        
    
    NA_M0=(1-naM0)
    

    
    
    
    ### FRAGMENT COMPOSITION
    df["lost_C"]=(df['parent.carbons']-df['daughter.carbons'])
    df["lost_H"]=(df['parent.hydrogen']-df['daughter.hydrogen'])
    df["lost_O"]=(df['parent.oxygen']-df['daughter.oxygen'])
    df["lost_N"]=(df['parent.nitrogen']-df['daughter.nitrogen'])
    
    LostFrag = ["lost_C","lost_H","lost_N","lost_O"]
    DaugFrag = ['daughter.carbons','daughter.hydrogen','daughter.nitrogen','daughter.oxygen']
    
    
    
    ###### M+1 ISOTOPOLOGUE
    mP1D0=df[(df['mass']=='M+1')&(df['daughter.label']==0)]
    mP1D1=df[(df['mass']=='M+1')&(df['daughter.label']==1)]
    
    
    # NA
    NA_P1D0 = sum(L1[i]*mP1D0[LostFrag[i]] for i in range(len(L1)))    
    NA_P1D1 = sum(L1[i]*mP1D1[DaugFrag[i]] for i in range(len(L1)))
   


    
    
    
    ###### M+2 ISOTOPOLOGUE
    mP2D0=df[(df['mass']=='M+2')&(df['daughter.label']==0)]
    mP2D1=df[(df['mass']=='M+2')&(df['daughter.label']==1)]
    mP2D2=df[(df['mass']=='M+2')&(df['daughter.label']==2)]

    
   
    
    NA_P2D0 = sum((L1[i]**2)*mP2D0[LostFrag[i]].apply(lambda x: combi(x,2)) for i in range(len(L1))) \
    + (na18O*mP2D0["lost_O"]) + sum(L1[i]*L1[j]*mP2D0[LostFrag[i]]*mP2D0[LostFrag[j]] for i,j in combinations(range(4),2))
    
    
    NA_P2D1 = sum((L1[i]**2)*mP2D1[LostFrag[i]]*mP2D1[DaugFrag[i]] for i in range(len(L1))) \
     + sum(L1[i]*L1[j]*((mP2D1[LostFrag[i]]*mP2D1[DaugFrag[j]])+(mP2D1[DaugFrag[i]]*mP2D1[LostFrag[j]])) for i,j in combinations(range(4),2))
 
    NA_P2D2 = sum((L1[i]**2)*mP2D2[DaugFrag[i]].apply(lambda x: combi(x,2)) for i in range(len(L1))) \
    + (na18O*mP2D2["daughter.oxygen"]) + sum(L1[i]*L1[j]*mP2D2[DaugFrag[i]]*mP2D2[DaugFrag[j]] for i,j in combinations(range(4),2))

    
    
        
    
    #M+3 and higher have no significant NA
    metM3plus=df[(df['parent.label']>=3)]
    NA3plus = pd.Series(0, index=metM3plus.index) # In the case of M+3 and beyond, the natural abundance are negligible and thus NA was set to 0.
    
    
    
    NA=pd.concat([NA_M0,NA_P1D0,NA_P1D1,NA_P2D0,NA_P2D1,NA_P2D2,NA3plus])
    return NA



    

    
    
# Natural abundance calculation
NA = natural_abundance(m_df.dropna())

# Merging Natural abundance value into the metabolite info dataframe
m_df["NA"] = NA

<ipython-input-9-0e6de4e63e08>:73: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["lost_C"]=(df['parent.carbons']-df['daughter.carbons'])
<ipython-input-9-0e6de4e63e08>:74: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["lost_H"]=(df['parent.hydrogen']-df['daughter.hydrogen'])
<ipython-input-9-0e6de4e63e08>:75: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://

In [10]:
# Read each file into a pandas dataframe
s_dfs = []
for filename in s_files:
    df = pd.read_csv(filename,index_col=['Original.Sample.Name'])
    s_dfs.append(df)

# Concatenate all dataframes into one sampleinfo dataframe
s_df = pd.concat(s_dfs)

# List of sample names without Stds from sampleinfo df
df_filtered = s_df[~s_df['GroupLevel1'].str.contains('std')]
df_stds = s_df[s_df['GroupLevel1'].str.contains('std')]
sample_list=set(df_filtered.index.tolist())

# Opens each Area file
a_dfs = []
for filename in a_files:
    # Read Area file into a pandas dataframe
    df = pd.read_csv(filename).set_index('compoundId')
    a_dfs.append(df)
a_df = pd.concat(a_dfs)

### NA correction and Enrichment calculation

In [11]:
# Opens each Area file
for filename in a_files:
    # Read Area file into a pandas dataframe
    df = pd.read_csv(filename).set_index('compoundId')

    # Filtered area df rows based on sample name list
    new_a_df = df.loc[:, df.columns.isin(sample_list)]
    combined_df = pd.concat([new_a_df,m_df[["metabolite","mass","NA"]]], axis=1, join="inner").reset_index().set_index(["compoundId","metabolite","mass"])
    odf = combined_df.copy().drop('NA', axis=1)

    # SumArea combined area df
    sum_df = odf.groupby(level=[1]).transform("sum")

    # NA correction
    # M+0
    filtered_M0 = odf.index.get_level_values('mass') == 'M+0'
    odf.loc[filtered_M0] = odf.loc[filtered_M0].div(combined_df["NA"], axis="index")

    #>=M+1
    filtered_Mplus = odf.index.get_level_values('mass') != 'M+0'
    sum_df.loc[filtered_Mplus]  = sum_df.loc[filtered_Mplus].mul(combined_df["NA"], axis="index")

    odf.loc[filtered_Mplus] = odf.loc[filtered_Mplus].sub(sum_df.loc[filtered_Mplus])
    odf = odf.clip(lower=0)

    # NA Corrected Area and Enrichment calculation
    sum_odf = odf.groupby(level=1).transform('sum')
    result = (odf/sum_odf).droplevel([1,2]).T
    result.index = result.index.str.lower()
    
    
    
    # Write Enrichment into a csv file
    result.to_csv("Enrichments_from_"+filename, index=True, index_label="compoundid")